In [31]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [32]:
pizza = pd.read_csv('Pizzasales.csv')
ingredients = pd.read_csv('Pizzaingredients.csv')

NEXT ONE WEEK ORDERDATE AND QUANTITY

In [33]:
pizza.set_index('order_date', inplace=True)

train = pizza['quantity'][:-7]  
test = pizza['quantity'][-7:] 

model_arima = ARIMA(train, order=(5, 1, 0))
model_arima_fit = model_arima.fit()

forecast_arima = model_arima_fit.forecast(steps=7)

last_date = train.index[-1]  
future_dates = pd.date_range(start=last_date, periods=8, freq='D')[1:] 

forecast_df = pd.DataFrame({
    'order_date': future_dates,
    'quantity': forecast_arima
})

print("Predicted Sales for the Next Week:")
print(forecast_df)

Predicted Sales for the Next Week:
      order_date  quantity
48613 2016-01-01       1.0
48614 2016-01-02       1.0
48615 2016-01-03       1.0
48616 2016-01-04       1.0
48617 2016-01-05       1.0
48618 2016-01-06       1.0
48619 2016-01-07       1.0


SALES FORECASTING

In [42]:
pizza_df = pizza[['order_date', 'pizza_name', 'quantity']]

In [43]:
sales_summary = pizza_df.groupby(['order_date', 'pizza_name']).sum().reset_index()

In [44]:
sales_summary.head()

,order_date,pizza_name,quantity
0,2015-01-01,The Barbecue Chicken Pizza,11
1,2015-01-01,The Big Meat Pizza,5
2,2015-01-01,The Calabrese Pizza,1
3,2015-01-01,The California Chicken Pizza,4
4,2015-01-01,The Chicken Alfredo Pizza,2


In [45]:
sales_pivot = sales_summary.pivot(index='order_date', columns='pizza_name', values='quantity').fillna(0)

In [46]:
arima_models = {}

for pizza_name in sales_pivot.columns:
  try:
    model = ARIMA(sales_pivot[pizza_name], order=(1, 1, 0))
    model_fit = model.fit()
    arima_models[pizza_name] = model_fit
  except:
    print(f'ARIMA model for {pizza_name} failed to fit')

In [47]:
prediction_days = 7
predictions_arima = {}

for pizza_name, model in arima_models.items():
  predictions_arima[pizza_name] = model.predict(start=len(sales_pivot), end=len(sales_pivot) + prediction_days - 1)

In [48]:
predictions_df = pd.DataFrame(predictions_arima)

In [49]:
predictions_df.index = pd.date_range(start=sales_pivot.index[-1], periods=prediction_days, freq='D')

In [50]:
ingredients.rename(columns={'Items_Qty_In_Grams': 'items_qty'}, inplace=True)

In [51]:
predictions_df

,The Barbecue Chicken Pizza,The Big Meat Pizza,The Brie Carre Pizza,The Calabrese Pizza,The California Chicken Pizza,The Chicken Alfredo Pizza,The Chicken Pesto Pizza,The Classic Deluxe Pizza,The Five Cheese Pizza,The Four Cheese Pizza,...,The Sicilian Pizza,The Soppressata Pizza,The Southwest Chicken Pizza,The Spicy Italian Pizza,The Spinach Pesto Pizza,The Spinach Supreme Pizza,The Spinach and Feta Pizza,The Thai Chicken Pizza,The Vegetables + Vegetables Pizza,Unknown
2015-12-31,7.940297,4.420315,0.868596,1.368239,7.0,2.529125,2.139860,6.884102,3.033661,6.965904,...,2.583928,2.064211,5.918578,5.804710,3.537620,2.0,3.169020,3.0,7.773728,0.0
2016-01-01,9.000891,5.252117,1.508634,2.255787,7.0,3.250283,3.004890,8.502239,4.000283,8.000291,...,3.751409,2.502062,5.496685,7.263264,4.250472,2.0,4.007142,3.0,8.764985,0.0
2016-01-02,8.454764,4.814122,1.146563,1.773032,7.0,2.896705,2.602621,7.661914,3.525106,7.474280,...,3.187265,2.297193,5.690456,6.597478,3.902985,2.0,3.623496,3.0,8.323624,0.0
2016-01-03,8.735979,5.044753,1.351387,2.035612,7.0,3.070061,2.789690,8.098308,3.758696,7.741769,...,3.459867,2.393050,5.601459,6.901389,4.072371,2.0,3.799108,3.0,8.520142,0.0
2016-01-04,8.591174,4.923311,1.235518,1.892789,7.0,2.985066,2.702696,7.871682,3.643867,7.605745,...,3.328142,2.348199,5.642335,6.762663,3.989802,2.0,3.718722,3.0,8.432641,0.0
2016-01-05,8.665738,4.987258,1.301065,1.970474,7.0,3.026739,2.743152,7.989373,3.700315,7.674916,...,3.391794,2.369185,5.623561,6.825988,4.030051,2.0,3.755518,3.0,8.471601,0.0
2016-01-06,8.627343,4.953586,1.263985,1.928219,7.0,3.006307,2.724338,7.928254,3.672566,7.639741,...,3.361036,2.359366,5.632183,6.797082,4.010431,2.0,3.738675,3.0,8.454254,0.0


INGREDIENTS CALCULATION

In [53]:
ingredients.drop(columns=['Unnamed: 0'], inplace=True)

In [54]:
ingredients.head()

,pizza_name_id,pizza_name,pizza_ingredients,items_qty
0,bbq_ckn_l,The Barbecue Chicken Pizza,Barbecued Chicken,40.0
1,bbq_ckn_l,The Barbecue Chicken Pizza,Red Peppers,15.0
2,bbq_ckn_l,The Barbecue Chicken Pizza,Green Peppers,20.0
3,bbq_ckn_l,The Barbecue Chicken Pizza,Tomatoes,30.0
4,bbq_ckn_l,The Barbecue Chicken Pizza,Red Onions,60.0


In [55]:
ingredient_quantities = {}

for pizza_name in predictions_df.columns:

  predicted_quantity = predictions_df[pizza_name].sum()

  pizza_ingredients = ingredients[ingredients['pizza_name'] == pizza_name]

  for index, row in pizza_ingredients.iterrows():
    ingredient = row['pizza_ingredients']
    ingredient_qty = row['items_qty']

    required_quantity = predicted_quantity * ingredient_qty

    if ingredient not in ingredient_quantities:
      ingredient_quantities[ingredient] = 0
    ingredient_quantities[ingredient] += required_quantity

In [56]:
ingredient_requirements_df = pd.DataFrame.from_dict(ingredient_quantities, orient='index', columns=['required_quantity'])

In [57]:
ingredient_requirements_df

,required_quantity
Barbecued Chicken,5401.456838
Red Peppers,11339.700349
Green Peppers,8028.578723
Tomatoes,34970.259430
Red Onions,54801.315406
...,...
Luganega Sausage,2767.612911
Onions,1383.806456
Soppressata Salami,4899.979682
Peperoncini verdi,1408.577241


PURCHASE ORDER CREATION

In [61]:
purchase_order_df = ingredient_requirements_df.copy()

purchase_order_df['unit'] = 'grams'

purchase_order_df = purchase_order_df.rename(columns={'required_quantity': 'quantity'})

print(purchase_order_df.to_string())


                                quantity   unit
Barbecued Chicken            5401.456838  grams
Red Peppers                 11339.700349  grams
Green Peppers                8028.578723  grams
Tomatoes                    34970.259430  grams
Red Onions                  54801.315406  grams
Barbecue Sauce               1800.485613  grams
Bacon                       19992.004764  grams
Pepperoni                   24186.495331  grams
Italian Sausage               343.954622  grams
Chorizo Sausage              1719.773109  grams
Brie Carre Cheese             260.272404  grams
Prosciutto                    260.272404  grams
Caramelized Onions            173.514936  grams
Pears                          86.757468  grams
Thyme                          43.378734  grams
Garlic                      17931.797062  grams
?duja Salami                 1586.898271  grams
Pancetta                     2380.347406  grams
Friggitello Peppers           396.724568  grams
Chicken                     44723.037831